In [ ]:
!pip install requests musicbrainzngs

In [ ]:
#Masukin ke kolom terakhir best MBID nya
import requests
import musicbrainzngs
import pandas as pd
import re

# Setup MusicBrainz API
musicbrainzngs.set_useragent("YourAppName", "1.0", "primadwi84@gmail.com")

# Fungsi untuk normalisasi judul lagu (mengatasi masalah tanda petik dan karakter khusus)
def normalize_title(title):
    title = title.lower()
    title = re.sub(r"[’']", "'", title)  # Ganti semua variasi tanda petik dengan petik tunggal standar
    title = re.sub(r"[^a-z0-9 ]", "", title)  # Hapus karakter non-alfanumerik kecuali spasi
    return title

# Fungsi untuk mengecek ketersediaan data di AcousticBrainz
def get_acousticbrainz_data_availability(mbid, expected_title):
    high_level_url = f"https://acousticbrainz.org/{mbid}/high-level"
    response = requests.get(high_level_url)

    if response.status_code == 200 and response.json():
        metadata_url = f"https://acousticbrainz.org/{mbid}"
        metadata_response = requests.get(metadata_url)
        if metadata_response.status_code == 200:
            page_content = metadata_response.text.lower()
            normalized_expected_title = normalize_title(expected_title)
            normalized_page_content = normalize_title(page_content)
            if normalized_expected_title in normalized_page_content:
                return True
    return False

# Fungsi untuk mendapatkan MBID berdasarkan input artist dan judul lagu
def get_mbid_for_track_with_exact_title_and_acoustic_data(artist_name, track_title, limit=20):
    result = musicbrainzngs.search_recordings(artist=artist_name, recording=track_title, limit=limit)
    print(f"Search results for '{track_title}' by '{artist_name}': {len(result['recording-list'])} recordings found")

    valid_mbids = []
    normalized_track_title = normalize_title(track_title)
    for recording in result['recording-list']:
        mbid = recording['id']
        recording_title = recording.get('title', 'Unknown Title').lower()
        releases = recording.get('release-list', [])

        for release in releases:
            release_group_type = release.get('release-group', {}).get('type', 'Unknown Type')

            if get_acousticbrainz_data_availability(mbid, normalized_track_title):
                valid_mbids.append(mbid)

    if valid_mbids:
        best_mbid = valid_mbids[0]
        return best_mbid

    return None

# Fungsi baru untuk memproses CSV dan menambahkan kolom 'Best MBID'
def get_mbids_from_csv(file_path, max_rows=None, output_file='output_with_mbids.csv'):
    df = pd.read_csv(file_path)
    df['Best MBID'] = None
    for index, row in df.head(max_rows).iterrows():
        artist_name = row['Artist']
        track_title = row['Cleaned Title']
        mbid = get_mbid_for_track_with_exact_title_and_acoustic_data(artist_name, track_title)
        df.at[index, 'Best MBID'] = mbid

    # Simpan hasilnya ke CSV baru
    df.to_csv(output_file, index=False)
    print(f"CSV baru telah disimpan dengan nama: {output_file}")

# Contoh penggunaan
csv_file_path = './clean_single_songs_part_1.csv'
get_mbids_from_csv(csv_file_path, max_rows=5, output_file='output_with_mbids.csv')

Search results for 'I Don't Want To Miss A Thing' by 'Aerosmith': 20 recordings found
Search results for 'The First Night' by 'Monica': 20 recordings found
Search results for 'Crush' by 'Jennifer Paige': 20 recordings found
Search results for 'My Way' by 'Usher': 20 recordings found
Search results for 'The Boy Is Mine' by 'Brandy & Monica': 20 recordings found
CSV baru telah disimpan dengan nama: output_with_mbids.csv


In [ ]:
the_mbid = []

In [ ]:
#BottleNeck ini yg berhasil 100 data dengan 5 kasus ga nemu
import requests
import musicbrainzngs
import pandas as pd
import re
from time import sleep

# Setup MusicBrainz API
musicbrainzngs.set_useragent("YourAppName", "1.0", "primadwi84@gmail.com")

# Fungsi untuk normalisasi judul lagu (mengatasi masalah tanda petik dan karakter khusus)
def normalize_title(title):
    title = title.lower()
    title = re.sub(r"[’']", "'", title)  # Ganti semua variasi tanda petik dengan petik tunggal standar
    title = re.sub(r"[^a-z0-9 ]", "", title)  # Hapus karakter non-alfanumerik kecuali spasi
    return title

# Fungsi untuk mengecek ketersediaan data di AcousticBrainz
def get_acousticbrainz_data_availability(mbid, expected_title):
    high_level_url = f"https://acousticbrainz.org/{mbid}/high-level"
    response = requests.get(high_level_url)

    if response.status_code == 200 and response.json():
        metadata_url = f"https://acousticbrainz.org/{mbid}"
        metadata_response = requests.get(metadata_url)
        if metadata_response.status_code == 200:
            page_content = metadata_response.text.lower()
            normalized_expected_title = normalize_title(expected_title)
            normalized_page_content = normalize_title(page_content)
            if normalized_expected_title in normalized_page_content:
                return True
    return False

# Fungsi untuk mendapatkan MBID berdasarkan input artist dan judul lagu
def get_mbid_for_track_with_dynamic_iteration(artist_name, track_title, max_limit=200, step=20):
    normalized_track_title = normalize_title(track_title)
    valid_mbids = []

    for limit in range(step, max_limit + step, step):
        try:
            result = musicbrainzngs.search_recordings(artist=artist_name, recording=track_title, limit=limit)

            print(f"Search results (limit={limit}) for '{track_title}' by '{artist_name}': {len(result['recording-list'])} recordings found")

            for recording in result['recording-list']:
                mbid = recording['id']

                if get_acousticbrainz_data_availability(mbid, normalized_track_title):
                    return mbid  # Langsung return jika ditemukan MBID yang valid

            # Jika sudah mencapai limit tertinggi dan belum menemukan, berhenti
            if len(result['recording-list']) < limit:
                break

        except Exception as e:
            print(f"Error during MBID search: {e}")
            sleep(1)  # Tunggu sebentar jika terjadi error

    return None

# Fungsi untuk memproses CSV dan menambahkan kolom 'Best MBID'
def get_mbids_from_csv(file_path, max_rows=None, output_file='output_with_mbids.csv'):
    df = pd.read_csv(file_path)
    df['Best MBID'] = None
    for index, row in df.head(max_rows).iterrows():
        artist_name = row['Main Artist']
        track_title = row['Cleaned Title']
        mbid = get_mbid_for_track_with_dynamic_iteration(artist_name, track_title)
        the_mbid.append(mbid)
        df.at[index, 'Best MBID'] = mbid
        print(f"Processed {index + 1}/{len(df)}: {track_title} - {mbid}")

    # Simpan hasilnya ke CSV baru
    df.to_csv(output_file, index=False)
    print(f"CSV baru telah disimpan dengan nama: {output_file}")

# Contoh penggunaan
csv_file_path = './clean_single_songs_part_12.csv'
get_mbids_from_csv(csv_file_path, max_rows=451, output_file='output_with_mbids_12.csv')

Search results (limit=20) for 'Should've Wore A Bonnet' by '21 Savage': 20 recordings found
Search results (limit=40) for 'Should've Wore A Bonnet' by '21 Savage': 40 recordings found
Search results (limit=60) for 'Should've Wore A Bonnet' by '21 Savage': 60 recordings found
Search results (limit=80) for 'Should've Wore A Bonnet' by '21 Savage': 80 recordings found
Search results (limit=100) for 'Should've Wore A Bonnet' by '21 Savage': 100 recordings found
Search results (limit=120) for 'Should've Wore A Bonnet' by '21 Savage': 25 recordings found
Processed 1/451: Should've Wore A Bonnet - None
Search results (limit=20) for 'Prove It' by '21 Savage': 20 recordings found
Search results (limit=40) for 'Prove It' by '21 Savage': 40 recordings found
Search results (limit=60) for 'Prove It' by '21 Savage': 60 recordings found
Search results (limit=80) for 'Prove It' by '21 Savage': 80 recordings found
Search results (limit=100) for 'Prove It' by '21 Savage': 100 recordings found
Search res

KeyboardInterrupt: 

In [ ]:
import requests
import musicbrainzngs
import pandas as pd
import re
from time import sleep

# Setup MusicBrainz API
musicbrainzngs.set_useragent("YourAppName", "1.0", "primadwi84@gmail.com")

# Fungsi untuk normalisasi judul lagu (mengatasi masalah tanda petik dan karakter khusus)
def normalize_title(title):
    title = title.lower()
    title = re.sub(r"[’']", "'", title)  # Ganti semua variasi tanda petik dengan petik tunggal standar
    title = re.sub(r"[^a-z0-9 ]", "", title)  # Hapus karakter non-alfanumerik kecuali spasi
    return title

# Fungsi untuk mengecek ketersediaan data di AcousticBrainz
def get_acousticbrainz_data_availability(mbid, expected_title):
    high_level_url = f"https://acousticbrainz.org/{mbid}/high-level"
    response = requests.get(high_level_url)

    if response.status_code == 200 and response.json():
        metadata_url = f"https://acousticbrainz.org/{mbid}"
        metadata_response = requests.get(metadata_url)
        if metadata_response.status_code == 200:
            page_content = metadata_response.text.lower()
            normalized_expected_title = normalize_title(expected_title)
            normalized_page_content = normalize_title(page_content)
            if normalized_expected_title in normalized_page_content:
                return True
    return False

# Fungsi untuk mendapatkan MBID berdasarkan input artist dan judul lagu
def get_mbid_for_track_with_dynamic_iteration(artist_name, track_title, max_limit=200, step=20):
    normalized_track_title = normalize_title(track_title)
    valid_mbids = []

    for limit in range(step, max_limit + step, step):
        try:
            result = musicbrainzngs.search_recordings(artist=artist_name, recording=track_title, limit=limit)

            print(f"Search results (limit={limit}) for '{track_title}' by '{artist_name}': {len(result['recording-list'])} recordings found")

            for recording in result['recording-list']:
                mbid = recording['id']

                if get_acousticbrainz_data_availability(mbid, normalized_track_title):
                    return mbid  # Langsung return jika ditemukan MBID yang valid

            # Jika sudah mencapai limit tertinggi dan belum menemukan, berhenti
            if len(result['recording-list']) < limit:
                break

        except Exception as e:
            print(f"Error during MBID search: {e}")
            sleep(1)  # Tunggu sebentar jika terjadi error

    return None

# Fungsi untuk memproses CSV dan menambahkan kolom 'Best MBID' dengan range baris tertentu
def get_mbids_from_csv(file_path, start_row=0, end_row=None, output_file='output_with_mbids.csv'):
    df = pd.read_csv(file_path)
    df['Best MBID'] = None

    # Batasi dataframe sesuai dengan range baris yang diinginkan
    if end_row is None or end_row > len(df):
        end_row = len(df)

    # Iterasi hanya pada baris yang diinginkan
    for index, row in df.iloc[start_row:end_row].iterrows():
        artist_name = row['Main Artist']
        track_title = row['Cleaned Title']
        mbid = get_mbid_for_track_with_dynamic_iteration(artist_name, track_title)
        the_mbid.append(mbid)
        df.at[index, 'Best MBID'] = mbid
        print(f"Processed {index + 1}/{len(df)}: {track_title} - {mbid}")

    # Simpan hasilnya ke CSV baru
    df.to_csv(output_file, index=False)
    print(f"CSV baru telah disimpan dengan nama: {output_file}")

# Contoh penggunaan
csv_file_path = './clean_single_songs_part_5.csv'
get_mbids_from_csv(csv_file_path, start_row=509, end_row=1000, output_file='output_with_mbids.csv')

Search results (limit=20) for 'Tik Tok' by 'Glee Cast': 20 recordings found
Processed 510/1000: Tik Tok - 9c33f2ce-1d36-4be2-92d0-1c7d423bd684
Search results (limit=20) for 'Someone Like You' by 'Adele': 20 recordings found
Processed 511/1000: Someone Like You - aaea0060-e286-4ca8-b216-f80d381264ab
Search results (limit=20) for 'Boyfriend' by 'Big Time Rush': 20 recordings found
Processed 512/1000: Boyfriend - a9e8ee80-0a5c-43fe-bf9e-bf3db6f6808f
Search results (limit=20) for 'Bow Chicka Wow Wow' by 'Mike Posner': 20 recordings found
Processed 513/1000: Bow Chicka Wow Wow - f13af21e-5a1a-4050-b3c7-3d422b801e25
Search results (limit=20) for 'Set Fire To The Rain' by 'Adele': 20 recordings found
Processed 514/1000: Set Fire To The Rain - 0c47fc76-04e0-452b-bb33-bc66d94c1968
Search results (limit=20) for 'Welcome To My Hood' by 'DJ Khaled': 20 recordings found
Processed 515/1000: Welcome To My Hood - a75dc8ce-8d06-4dd0-b92f-c366c0684789
Search results (limit=20) for 'I Smile' by 'Kirk Fra

In [ ]:
df = pd.DataFrame(the_mbid, columns=['MBID'])

df.to_csv('mbid.csv', index=False, encoding='utf-8')

In [ ]:
#dari Bottleneck, normalisasi tidak hanya title tapi juga artis tapi ga ngaruh
import requests
import musicbrainzngs
import pandas as pd
import re
from time import sleep

# Setup MusicBrainz API
musicbrainzngs.set_useragent("YourAppName", "1.0", "primadwi84@gmail.com")

# Fungsi untuk normalisasi judul lagu dan nama artis
def normalize_title(title):
    title = title.lower()
    title = re.sub(r"[’']", "'", title)  # Ganti semua variasi tanda petik dengan petik tunggal standar
    title = re.sub(r"[^a-z0-9 ]", "", title)  # Hapus karakter non-alfanumerik kecuali spasi
    return title

# Fungsi untuk mengecek ketersediaan data di AcousticBrainz
def get_acousticbrainz_data_availability(mbid, expected_title):
    high_level_url = f"https://acousticbrainz.org/{mbid}/high-level"
    response = requests.get(high_level_url)

    if response.status_code == 200 and response.json():
        metadata_url = f"https://acousticbrainz.org/{mbid}"
        metadata_response = requests.get(metadata_url)
        if metadata_response.status_code == 200:
            page_content = metadata_response.text.lower()
            normalized_expected_title = normalize_title(expected_title)
            normalized_page_content = normalize_title(page_content)
            if normalized_expected_title in normalized_page_content:
                return True
    return False

# Fungsi untuk mendapatkan MBID berdasarkan input artist dan judul lagu
def get_mbid_for_track_with_dynamic_iteration(artist_name, track_title, max_limit=200, step=20):
    normalized_artist_name = normalize_title(artist_name)  # Normalisasi nama artis
    normalized_track_title = normalize_title(track_title)
    valid_mbids = []

    for limit in range(step, max_limit + step, step):
        try:
            result = musicbrainzngs.search_recordings(artist=normalized_artist_name, recording=track_title, limit=limit)
            print(f"Search results (limit={limit}) for '{track_title}' by '{artist_name}': {len(result['recording-list'])} recordings found")

            for recording in result['recording-list']:
                mbid = recording['id']

                if get_acousticbrainz_data_availability(mbid, normalized_track_title):
                    return mbid  # Langsung return jika ditemukan MBID yang valid

            # Jika sudah mencapai limit tertinggi dan belum menemukan, berhenti
            if len(result['recording-list']) < limit:
                break

        except Exception as e:
            print(f"Error during MBID search: {e}")
            sleep(1)  # Tunggu sebentar jika terjadi error

    return None

# # Fungsi untuk memproses CSV dan menambahkan kolom 'Best MBID'
# def get_mbids_from_csv(file_path, max_rows=None, output_file='output_with_mbids.csv'):
#     df = pd.read_csv(file_path)
#     df['Best MBID'] = None
#     for index, row in df.head(max_rows).iterrows():
#         artist_name = row['Artist']
#         track_title = row['Cleaned Title']
#         mbid = get_mbid_for_track_with_dynamic_iteration(artist_name, track_title)
#         df.at[index, 'Best MBID'] = mbid
#         print(f"Processed {index + 1}/{len(df)}: {track_title} - {mbid}")

#     # Simpan hasilnya ke CSV baru
#     df.to_csv(output_file, index=False)
#     print(f"CSV baru telah disimpan dengan nama: {output_file}")

# # Contoh penggunaan
# csv_file_path = './clean_single_songs_part_1.csv'
# get_mbids_from_csv(csv_file_path, max_rows=10, output_file='output_with_mbids.csv')

mbid = get_mbid_for_track_with_dynamic_iteration("Aerosmith", "I Don't Want To Miss A Thing")
print(f"MBID terpilih : {mbid}")

mbid = get_mbid_for_track_with_dynamic_iteration("Cam'Ron", "Horse & Carriage")
print(f"MBID terpilih : {mbid}")

mbid = get_mbid_for_track_with_dynamic_iteration("J'Son", "I Should Cheat On You")
print(f"MBID terpilih : {mbid}")

mbid = get_mbid_for_track_with_dynamic_iteration("Athenaeum", "What I Didn't Know")
print(f"MBID terpilih : {mbid}")

mbid = get_mbid_for_track_with_dynamic_iteration("Tragedy", "'98 Thug Paradise")
print(f"MBID terpilih : {mbid}")

mbid = get_mbid_for_track_with_dynamic_iteration("Various Artists", "One Heart At A Time")
print(f"MBID terpilih : {mbid}")

Search results (limit=20) for 'I Don't Want To Miss A Thing' by 'Aerosmith': 20 recordings found
MBID terpilih : d9f551a9-6e48-4850-a0cb-ba3d38ef3dfa
Search results (limit=20) for 'Horse & Carriage' by 'Cam'Ron': 20 recordings found
Search results (limit=40) for 'Horse & Carriage' by 'Cam'Ron': 40 recordings found
Search results (limit=60) for 'Horse & Carriage' by 'Cam'Ron': 60 recordings found
Search results (limit=80) for 'Horse & Carriage' by 'Cam'Ron': 80 recordings found
Search results (limit=100) for 'Horse & Carriage' by 'Cam'Ron': 100 recordings found
Search results (limit=120) for 'Horse & Carriage' by 'Cam'Ron': 25 recordings found
MBID terpilih : None
Search results (limit=20) for 'I Should Cheat On You' by 'J'Son': 20 recordings found
Search results (limit=40) for 'I Should Cheat On You' by 'J'Son': 40 recordings found
Search results (limit=60) for 'I Should Cheat On You' by 'J'Son': 60 recordings found
Search results (limit=80) for 'I Should Cheat On You' by 'J'Son': 80 r

In [ ]:
import requests
import musicbrainzngs
import re
from time import sleep

# Setup MusicBrainz API
musicbrainzngs.set_useragent("YourAppName", "1.0", "primadwi84@gmail.com")

# Fungsi untuk normalisasi judul lagu
def normalize_title(title):
    if not title:
        return ""
    title = title.lower()
    title = re.sub(r"[’']", "'", title)  # Ganti tanda petik
    title = re.sub(r"[^a-z0-9 ]", "", title)  # Hapus karakter non-alfanumerik kecuali spasi
    return title.strip()

# Fungsi untuk mengecek ketersediaan data di AcousticBrainz
def get_acousticbrainz_data_availability(mbid, expected_title):
    high_level_url = f"https://acousticbrainz.org/{mbid}/high-level"
    response = requests.get(high_level_url)

    if response.status_code == 200 and response.json():
        metadata_url = f"https://acousticbrainz.org/{mbid}"
        metadata_response = requests.get(metadata_url)
        if metadata_response.status_code == 200:
            page_content = metadata_response.text.lower()
            normalized_expected_title = normalize_title(expected_title)
            normalized_page_content = normalize_title(page_content)
            print(f"Comparing expected '{normalized_expected_title}' with page content...")

            if normalized_expected_title in normalized_page_content:
                return True
    return False

# Fungsi untuk mendapatkan MBID dengan iterasi dinamis
def get_mbid_for_track_with_dynamic_iteration(artist_name, track_title, max_limit=200, step=20):
    normalized_track_title = normalize_title(track_title)

    for limit in range(step, max_limit + step, step):
        try:
            result = musicbrainzngs.search_recordings(artist=artist_name, recording=track_title, limit=limit)

            print(f"Search results (limit={limit}) for '{track_title}' by '{artist_name}': {len(result['recording-list'])} recordings found")

            for recording in result['recording-list']:
                mbid = recording['id']
                recording_title = recording['title'] if 'title' in recording else ''
                normalized_recording_title = normalize_title(recording_title)

                print(f"Checking recording: {recording_title} (MBID: {mbid})")

                # Cek di AcousticBrainz jika judulnya cocok
                if normalized_recording_title == normalized_track_title:
                    if get_acousticbrainz_data_availability(mbid, normalized_track_title):
                        print(f"Valid MBID found: {mbid}")
                        return mbid

            if len(result['recording-list']) < limit:
                break

        except Exception as e:
            print(f"Error during MBID search: {e}")
            sleep(1)

    print(f"No valid MBID found for '{track_title}' by '{artist_name}'")
    return None

# Uji coba untuk lagu-lagu yang tidak ditemukan
mbid = get_mbid_for_track_with_dynamic_iteration("Cam'Ron", "Horse & Carriage")
print(f"MBID terpilih : {mbid}")

mbid = get_mbid_for_track_with_dynamic_iteration("J'Son", "I Should Cheat On You")
print(f"MBID terpilih : {mbid}")

mbid = get_mbid_for_track_with_dynamic_iteration("Athenaeum", "What I Didn't Know")
print(f"MBID terpilih : {mbid}")

mbid = get_mbid_for_track_with_dynamic_iteration("Tragedy", "'98 Thug Paradise")
print(f"MBID terpilih : {mbid}")

mbid = get_mbid_for_track_with_dynamic_iteration("Various Artists", "One Heart At A Time")
print(f"MBID terpilih : {mbid}")


Search results (limit=20) for 'Horse & Carriage' by 'Cam'Ron': 20 recordings found
Checking recording: Horse & Carriage (MBID: a8d89157-c3ea-4a17-861a-2b195375ca32)
Comparing expected 'horse  carriage' with page content...
Checking recording: Horse & Carriage (MBID: c36a6a04-31b4-44c5-896e-f561574277d3)
Checking recording: Horse & Carriage (MBID: 1abce929-2e46-48c9-990b-d48c9f595060)
Checking recording: Horse & Carriage (MBID: 7b21ceb1-5f32-493d-8a3c-19fe3baad94f)
Comparing expected 'horse  carriage' with page content...
Checking recording: Horse & Carriage (MBID: f8d254f4-a109-41e3-9e04-0317cdfda6b5)
Checking recording: Horse & Carriage (MBID: 244744a6-c260-4cf8-a67d-9b9eb30ee2a4)
Checking recording: Horse and Carriage (MBID: 0a23ce2f-827f-4d26-9288-0ab71634a6fb)
Checking recording: Horse and Carriage (MBID: 9344b92a-2105-4f55-bac1-a8dfc0b754e8)
Checking recording: Horse and Carriage (MBID: 6dce1da3-c4b5-42ff-aca1-71aee6e1a73d)
Checking recording: Horse & Carriage (MBID: c4f1ab7c-3f47

In [ ]:
import requests
import musicbrainzngs
import re
from time import sleep
from difflib import SequenceMatcher

# Setup MusicBrainz API
musicbrainzngs.set_useragent("YourAppName", "1.0", "primadwi84@gmail.com")

# Fungsi untuk normalisasi judul lagu
def normalize_title(title):
    if not title:
        return ""
    title = title.lower()
    title = re.sub(r"[’']", "'", title)  # Ganti tanda petik
    title = re.sub(r"[^a-z0-9 ]", " ", title)  # Hanya biarkan huruf, angka, dan spasi
    title = re.sub(r"\s+", " ", title)  # Ganti spasi ganda dengan spasi tunggal
    return title.strip()

# Fungsi fuzzy matching untuk judul lagu
def is_title_similar(expected_title, page_content, threshold=0.7):
    similarity = SequenceMatcher(None, expected_title, page_content).ratio()
    return similarity >= threshold

# Fungsi untuk mengecek apakah judul atau artis mengandung tanda baca (selain spasi)
def contains_punctuation(text):
    return bool(re.search(r"[^\w\s]", text))

# Fungsi untuk mendapatkan MBID dengan iterasi dinamis
def get_mbid_for_track_with_dynamic_iteration(artist_name, track_title, max_limit=200, step=20):
    normalized_track_title = normalize_title(track_title)
    normalized_artist_name = normalize_title(artist_name)

    for limit in range(step, max_limit + step, step):
        try:
            result = musicbrainzngs.search_recordings(artist=artist_name, recording=track_title, limit=limit)

            print(f"Search results (limit={limit}) for '{track_title}' by '{artist_name}': {len(result['recording-list'])} recordings found")

            for recording in result['recording-list']:
                mbid = recording['id']
                recording_title = recording['title'] if 'title' in recording else ''
                recording_artist = recording['artist-credit'][0]['artist']['name'] if 'artist-credit' in recording else ''

                normalized_recording_title = normalize_title(recording_title)
                normalized_recording_artist = normalize_title(recording_artist)

                print(f"Checking recording: '{recording_title}' by '{recording_artist}' (MBID: {mbid})")

                # Jika judul atau artis mengandung tanda baca, pilih langsung jika ada kecocokan kata
                if contains_punctuation(track_title) or contains_punctuation(artist_name):
                    expected_words = set(normalized_track_title.split() + normalized_artist_name.split())
                    actual_words = set(normalized_recording_title.split() + normalized_recording_artist.split())
                    matching_words = expected_words.intersection(actual_words)

                    print(f"Expected words: {expected_words}")
                    print(f"Actual words: {actual_words}")
                    print(f"Matching words: {matching_words}")

                    if matching_words:
                        print(f"Direct match found due to punctuation rule! MBID: {mbid}")
                        return mbid

                # Tetap gunakan fuzzy matching untuk kasus lainnya
                if is_title_similar(normalized_recording_title, normalized_track_title):
                    print(f"Fuzzy match found! MBID: {mbid}")
                    return mbid

            if len(result['recording-list']) < limit:
                break

        except Exception as e:
            print(f"Error during MBID search: {e}")
            sleep(1)

    print(f"No valid MBID found for '{track_title}' by '{artist_name}'")
    return None

# Uji coba untuk lagu-lagu yang tidak ditemukan sebelumnya
mbid = get_mbid_for_track_with_dynamic_iteration("Cam'Ron", "Horse & Carriage")
print(f"MBID terpilih : {mbid}")

mbid = get_mbid_for_track_with_dynamic_iteration("J'Son", "I Should Cheat On You")
print(f"MBID terpilih : {mbid}")

mbid = get_mbid_for_track_with_dynamic_iteration("Athenaeum", "What I Didn't Know")
print(f"MBID terpilih : {mbid}")

mbid = get_mbid_for_track_with_dynamic_iteration("Tragedy", "'98 Thug Paradise")
print(f"MBID terpilih : {mbid}")

mbid = get_mbid_for_track_with_dynamic_iteration("Various Artists", "One Heart At A Time")
print(f"MBID terpilih : {mbid}")


Search results (limit=20) for 'Horse & Carriage' by 'Cam'Ron': 20 recordings found
Checking recording: 'Horse & Carriage' by 'Cam’ron' (MBID: a8d89157-c3ea-4a17-861a-2b195375ca32)
Expected words: {'horse', 'ron', 'carriage', 'cam'}
Actual words: {'horse', 'ron', 'carriage', 'cam'}
Matching words: {'carriage', 'horse', 'ron', 'cam'}
Direct match found due to punctuation rule! MBID: a8d89157-c3ea-4a17-861a-2b195375ca32
MBID terpilih : a8d89157-c3ea-4a17-861a-2b195375ca32
Search results (limit=20) for 'I Should Cheat On You' by 'J'Son': 20 recordings found
Checking recording: 'I Should Cheat On You (Instrumental)' by 'J'son' (MBID: 196143e2-3239-4d05-8463-99f2df2a2642)
Expected words: {'should', 'on', 'j', 'cheat', 'son', 'you', 'i'}
Actual words: {'should', 'on', 'instrumental', 'j', 'cheat', 'son', 'you', 'i'}
Matching words: {'should', 'on', 'j', 'cheat', 'son', 'you', 'i'}
Direct match found due to punctuation rule! MBID: 196143e2-3239-4d05-8463-99f2df2a2642
MBID terpilih : 196143e2-3

In [ ]:
#handel 5 kasus
import requests
import musicbrainzngs
import pandas as pd
import re
from time import sleep

# Setup MusicBrainz API
musicbrainzngs.set_useragent("YourAppName", "1.0", "primadwi84@gmail.com")

# Fungsi untuk normalisasi judul lagu dan nama artis
def normalize_title(title):
    title = title.lower()
    title = re.sub(r"[’']", "'", title)  # Ganti semua variasi tanda petik dengan petik tunggal standar
    title = re.sub(r"[^a-z0-9 ]", "", title)  # Hapus karakter non-alfanumerik kecuali spasi
    return title

# Fungsi untuk mengecek ketersediaan data di AcousticBrainz
def get_acousticbrainz_data_availability(mbid, expected_title):
    high_level_url = f"https://acousticbrainz.org/{mbid}/high-level"
    response = requests.get(high_level_url)

    if response.status_code == 200 and response.json():
        metadata_url = f"https://acousticbrainz.org/{mbid}"
        metadata_response = requests.get(metadata_url)
        if metadata_response.status_code == 200:
            page_content = metadata_response.text.lower()
            normalized_expected_title = normalize_title(expected_title)
            normalized_page_content = normalize_title(page_content)
            if normalized_expected_title in normalized_page_content:
                return True
    return False

# Fungsi untuk mendapatkan MBID berdasarkan input artist dan judul lagu
def get_mbid_for_track_with_dynamic_iteration(artist_name, track_title, max_limit=200, step=20):
    normalized_track_title = normalize_title(track_title)
    normalized_artist_name = normalize_title(artist_name)
    valid_mbids = []

    for limit in range(step, max_limit + step, step):
        try:
            result = musicbrainzngs.search_recordings(artist=artist_name, recording=track_title, limit=limit)
            print(f"Search results (limit={limit}) for '{track_title}' by '{artist_name}': {len(result['recording-list'])} recordings found")

            for recording in result['recording-list']:
                mbid = recording['id']
                recording_title = recording.get('title', '').lower()
                release_type = recording.get('release-list', [{}])[0].get('release-group', {}).get('type', 'Unknown Type')

                # Check jika judul lagunya mirip atau memiliki acoustic data di AcousticBrainz
                if get_acousticbrainz_data_availability(mbid, normalized_track_title):
                    print(f"MBID found with acoustic data: {mbid}")
                    return mbid  # Langsung return jika ditemukan MBID yang valid

                # Alternatif jika versi judul berbeda (misal: Extended Version, Remix)
                if normalized_track_title in normalize_title(recording_title):
                    print(f"Alternative MBID found (title match): {mbid}")
                    return mbid

            # Jika sudah mencapai limit tertinggi dan belum menemukan, berhenti
            if len(result['recording-list']) < limit:
                break

        except Exception as e:
            print(f"Error during MBID search: {e}")
            sleep(1)  # Tunggu sebentar jika terjadi error

    return None

mbid = get_mbid_for_track_with_dynamic_iteration("Aerosmith", "I Don't Want To Miss A Thing")
print(f"MBID terpilih : {mbid}")

mbid = get_mbid_for_track_with_dynamic_iteration("Cam'Ron", "Horse & Carriage")
print(f"MBID terpilih : {mbid}")

mbid = get_mbid_for_track_with_dynamic_iteration("J'Son", "I Should Cheat On You")
print(f"MBID terpilih : {mbid}")

mbid = get_mbid_for_track_with_dynamic_iteration("Athenaeum", "What I Didn't Know")
print(f"MBID terpilih : {mbid}")


mbid = get_mbid_for_track_with_dynamic_iteration("Tragedy", "'98 Thug Paradise")
print(f"MBID terpilih : {mbid}")


mbid = get_mbid_for_track_with_dynamic_iteration("Various Artists", "One Heart At A Time")
print(f"MBID terpilih : {mbid}")

Search results (limit=20) for 'I Don't Want To Miss A Thing' by 'Aerosmith': 20 recordings found
Alternative MBID found (title match): 3a0be96f-e6b1-4a76-bb74-2314d102f341
MBID terpilih : 3a0be96f-e6b1-4a76-bb74-2314d102f341
Search results (limit=20) for 'Horse & Carriage' by 'Cam'Ron': 20 recordings found
Alternative MBID found (title match): a8d89157-c3ea-4a17-861a-2b195375ca32
MBID terpilih : a8d89157-c3ea-4a17-861a-2b195375ca32
Search results (limit=20) for 'I Should Cheat On You' by 'J'Son': 20 recordings found
Alternative MBID found (title match): 196143e2-3239-4d05-8463-99f2df2a2642
MBID terpilih : 196143e2-3239-4d05-8463-99f2df2a2642
Search results (limit=20) for 'What I Didn't Know' by 'Athenaeum': 20 recordings found
Alternative MBID found (title match): 92f9852c-7f6c-4edb-95d3-1e201280805e
MBID terpilih : 92f9852c-7f6c-4edb-95d3-1e201280805e
Search results (limit=20) for ''98 Thug Paradise' by 'Tragedy': 20 recordings found
Alternative MBID found (title match): b27da5c3-3dd8

In [ ]:
#lowlevel
import requests
import musicbrainzngs
import pandas as pd
import re
from time import sleep

# Setup MusicBrainz API
musicbrainzngs.set_useragent("YourAppName", "1.0", "primadwi84@gmail.com")

# Fungsi untuk normalisasi judul lagu dan nama artis
def normalize_title(title):
    title = title.lower()
    title = re.sub(r"[’']", "'", title)  # Ganti semua variasi tanda petik dengan petik tunggal standar
    title = re.sub(r"[^a-z0-9 ]", "", title)  # Hapus karakter non-alfanumerik kecuali spasi
    return title

# Fungsi untuk mengecek ketersediaan data di AcousticBrainz
def get_acousticbrainz_data_availability(mbid, expected_title):
    # Cek apakah ada data low-level (song features) di AcousticBrainz terlebih dahulu
    low_level_url = f"https://acousticbrainz.org/{mbid}/low-level"
    low_level_response = requests.get(low_level_url)

    if low_level_response.status_code == 200 and low_level_response.json():
        metadata_url = f"https://acousticbrainz.org/{mbid}"
        metadata_response = requests.get(metadata_url)

        if metadata_response.status_code == 200:
            page_content = metadata_response.text.lower()
            normalized_expected_title = normalize_title(expected_title)
            normalized_page_content = normalize_title(page_content)

            if normalized_expected_title in normalized_page_content:
                print(f"Song features found in low-level data for MBID: {mbid}")
                return True

    # Jika low-level tidak ada, coba cek high-level sebagai fallback
    high_level_url = f"https://acousticbrainz.org/{mbid}/high-level"
    high_level_response = requests.get(high_level_url)

    if high_level_response.status_code == 200 and high_level_response.json():
        metadata_url = f"https://acousticbrainz.org/{mbid}"
        metadata_response = requests.get(metadata_url)

        if metadata_response.status_code == 200:
            page_content = metadata_response.text.lower()
            normalized_expected_title = normalize_title(expected_title)
            normalized_page_content = normalize_title(page_content)

            if normalized_expected_title in normalized_page_content:
                print(f"Fallback to high-level data for MBID: {mbid}")
                return True

    return False

# Fungsi untuk mendapatkan MBID berdasarkan input artist dan judul lagu
def get_mbid_for_track_with_dynamic_iteration(artist_name, track_title, max_limit=200, step=20):
    normalized_track_title = normalize_title(track_title)
    normalized_artist_name = normalize_title(artist_name)
    valid_mbids = []

    for limit in range(step, max_limit + step, step):
        try:
            result = musicbrainzngs.search_recordings(artist=artist_name, recording=track_title, limit=limit)
            print(f"Search results (limit={limit}) for '{track_title}' by '{artist_name}': {len(result['recording-list'])} recordings found")

            for recording in result['recording-list']:
                mbid = recording['id']
                recording_title = recording.get('title', '').lower()
                release_type = recording.get('release-list', [{}])[0].get('release-group', {}).get('type', 'Unknown Type')

                # Check jika judul lagunya mirip atau memiliki acoustic data di AcousticBrainz
                if get_acousticbrainz_data_availability(mbid, normalized_track_title):
                    print(f"MBID found with acoustic data: {mbid}")
                    return mbid  # Langsung return jika ditemukan MBID yang valid

                # Alternatif jika versi judul berbeda (misal: Extended Version, Remix)
                if normalized_track_title in normalize_title(recording_title):
                    print(f"Alternative MBID found (title match): {mbid}")
                    return mbid

            # Jika sudah mencapai limit tertinggi dan belum menemukan, berhenti
            if len(result['recording-list']) < limit:
                break

        except Exception as e:
            print(f"Error during MBID search: {e}")
            sleep(1)  # Tunggu sebentar jika terjadi error

    return None

mbid = get_mbid_for_track_with_dynamic_iteration("Aerosmith", "I Don't Want To Miss A Thing")
print(f"MBID terpilih : {mbid}")

mbid = get_mbid_for_track_with_dynamic_iteration("Cam'Ron", "Horse & Carriage")
print(f"MBID terpilih : {mbid}")

mbid = get_mbid_for_track_with_dynamic_iteration("J'Son", "I Should Cheat On You")
print(f"MBID terpilih : {mbid}")

mbid = get_mbid_for_track_with_dynamic_iteration("Athenaeum", "What I Didn't Know")
print(f"MBID terpilih : {mbid}")


mbid = get_mbid_for_track_with_dynamic_iteration("Tragedy", "'98 Thug Paradise")
print(f"MBID terpilih : {mbid}")


mbid = get_mbid_for_track_with_dynamic_iteration("Various Artists", "One Heart At A Time")
print(f"MBID terpilih : {mbid}")

Search results (limit=20) for 'I Don't Want To Miss A Thing' by 'Aerosmith': 20 recordings found
Alternative MBID found (title match): 58587e90-3d72-485f-8d5f-e9d1b8cbb594
MBID terpilih : 58587e90-3d72-485f-8d5f-e9d1b8cbb594
Search results (limit=20) for 'Horse & Carriage' by 'Cam'Ron': 20 recordings found
Alternative MBID found (title match): 1abce929-2e46-48c9-990b-d48c9f595060
MBID terpilih : 1abce929-2e46-48c9-990b-d48c9f595060
Search results (limit=20) for 'I Should Cheat On You' by 'J'Son': 20 recordings found
Alternative MBID found (title match): 196143e2-3239-4d05-8463-99f2df2a2642
MBID terpilih : 196143e2-3239-4d05-8463-99f2df2a2642
Search results (limit=20) for 'What I Didn't Know' by 'Athenaeum': 20 recordings found
Alternative MBID found (title match): ac93a597-101f-4a02-876d-f5e081cca2fc
MBID terpilih : ac93a597-101f-4a02-876d-f5e081cca2fc
Search results (limit=20) for ''98 Thug Paradise' by 'Tragedy': 20 recordings found
Alternative MBID found (title match): b27da5c3-3dd8

In [ ]:
import requests
import musicbrainzngs
import pandas as pd
import re
from time import sleep

# Setup MusicBrainz API
musicbrainzngs.set_useragent("YourAppName", "1.0", "primadwi84@gmail.com")

# Fungsi untuk normalisasi judul lagu dan nama artis
def normalize_title(title):
    title = title.lower()
    title = re.sub(r"[’']", "'", title)  # Ganti semua variasi tanda petik dengan petik tunggal standar
    title = re.sub(r"[^a-z0-9 ]", "", title)  # Hapus karakter non-alfanumerik kecuali spasi
    return title

# Fungsi untuk mengecek ketersediaan data di AcousticBrainz
def get_acousticbrainz_data_availability(mbid, expected_title):
    high_level_url = f"https://acousticbrainz.org/{mbid}/high-level"
    response = requests.get(high_level_url)

    if response.status_code == 200 and response.json():
        metadata_url = f"https://acousticbrainz.org/{mbid}"
        metadata_response = requests.get(metadata_url)
        if metadata_response.status_code == 200:
            page_content = metadata_response.text.lower()
            normalized_expected_title = normalize_title(expected_title)
            normalized_page_content = normalize_title(page_content)
            if normalized_expected_title in normalized_page_content:
                return True
    return False

# Fungsi untuk mendapatkan MBID dengan berbagai skenario
def get_mbid_with_all_scenarios(artist_name, track_title, max_limit=200, step=20):
    normalized_track_title = normalize_title(track_title)
    valid_mbids = []

    for limit in range(step, max_limit + step, step):
        try:
            result = musicbrainzngs.search_recordings(artist=artist_name, recording=track_title, limit=limit)
            print(f"Search results (limit={limit}) for '{track_title}' by '{artist_name}': {len(result['recording-list'])} recordings found")

            for recording in result['recording-list']:
                mbid = recording['id']
                recording_title = recording.get('title', 'Unknown Title').lower()

                # Skenario 1: MBID dengan judul dan artis tepat + punya acoustic data
                if get_acousticbrainz_data_availability(mbid, normalized_track_title):
                    print(f"Skenario 1: MBID with exact match found: {mbid}")
                    return mbid

                # Skenario 2: Alternatif untuk versi Extended, Remix, dll.
                if normalized_track_title in normalize_title(recording_title):
                    if get_acousticbrainz_data_availability(mbid, recording_title):
                        print(f"Skenario 2: Alternative MBID (title match): {mbid}")
                        return mbid

                # Skenario 3: Ambil MBID apa pun dengan acoustic data (tanpa cek judul asli)
                if get_acousticbrainz_data_availability(mbid, recording_title):
                    print(f"Skenario 3: Any MBID with acoustic data: {mbid}")
                    valid_mbids.append(mbid)

            # Jika sudah mencapai limit tertinggi dan belum menemukan, berhenti
            if len(result['recording-list']) < limit:
                break

        except Exception as e:
            print(f"Error during MBID search: {e}")
            sleep(1)

    # Skenario 4: Fallback jika tidak ada yang sesuai, ambil MBID pertama dengan acoustic data
    if valid_mbids:
        print(f"Skenario 4: Fallback MBID chosen: {valid_mbids[0]}")
        return valid_mbids[0]

    # Skenario 5: Jika tidak ada MBID yang valid ditemukan sama sekali
    print("Skenario 5: No MBID found, returning 'Not Found'")
    return "Not Found"

# # Fungsi untuk memproses CSV dan menambahkan kolom 'Best MBID'
# def get_mbids_from_csv(file_path, max_rows=None, output_file='output_with_mbids.csv'):
#     df = pd.read_csv(file_path)
#     df['Best MBID'] = None

#     for index, row in df.head(max_rows).iterrows():
#         artist_name = row['Artist']
#         track_title = row['Cleaned Title']

#         mbid = get_mbid_with_all_scenarios(artist_name, track_title)
#         df.at[index, 'Best MBID'] = mbid
#         print(f"Processed {index + 1}/{len(df)}: {track_title} - {mbid}")

#     # Simpan hasilnya ke CSV baru
#     df.to_csv(output_file, index=False)
#     print(f"CSV baru telah disimpan dengan nama: {output_file}")

# # Contoh penggunaan
# csv_file_path = './clean_single_songs_part_1.csv'
# get_mbids_from_csv(csv_file_path, max_rows=100, output_file='output_with_mbids.csv')

mbid = get_mbid_for_track_with_dynamic_iteration("Aerosmith", "I Don't Want To Miss A Thing")
print(f"MBID terpilih : {mbid}")

mbid = get_mbid_for_track_with_dynamic_iteration("Cam'Ron", "Horse & Carriage")
print(f"MBID terpilih : {mbid}")

mbid = get_mbid_for_track_with_dynamic_iteration("J'Son", "I Should Cheat On You")
print(f"MBID terpilih : {mbid}")

mbid = get_mbid_for_track_with_dynamic_iteration("Athenaeum", "What I Didn't Know")
print(f"MBID terpilih : {mbid}")


mbid = get_mbid_for_track_with_dynamic_iteration("Tragedy", "'98 Thug Paradise")
print(f"MBID terpilih : {mbid}")


mbid = get_mbid_for_track_with_dynamic_iteration("Various Artists", "One Heart At A Time")
print(f"MBID terpilih : {mbid}")

Search results (limit=20) for 'I Don't Want To Miss A Thing' by 'Aerosmith': 20 recordings found
MBID terpilih : 563641a3-4479-402d-852b-c46aacb5c6e9
Search results (limit=20) for 'Horse & Carriage' by 'Cam'Ron': 20 recordings found
Search results (limit=40) for 'Horse & Carriage' by 'Cam'Ron': 40 recordings found
Search results (limit=60) for 'Horse & Carriage' by 'Cam'Ron': 60 recordings found
Search results (limit=80) for 'Horse & Carriage' by 'Cam'Ron': 80 recordings found
Search results (limit=100) for 'Horse & Carriage' by 'Cam'Ron': 100 recordings found
Search results (limit=120) for 'Horse & Carriage' by 'Cam'Ron': 25 recordings found
MBID terpilih : None
Search results (limit=20) for 'I Should Cheat On You' by 'J'Son': 20 recordings found
Search results (limit=40) for 'I Should Cheat On You' by 'J'Son': 40 recordings found
Search results (limit=60) for 'I Should Cheat On You' by 'J'Son': 60 recordings found
Search results (limit=80) for 'I Should Cheat On You' by 'J'Son': 80 r